In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
# import datetime
from datetime import date, datetime
from sqlalchemy import create_engine, MetaData # database connection
import numpy as np
import pandas as pd
import datetime as dt
from zipfile import ZipFile

# DB = {
#     'drivername': 'mysql',
#     'servername': '192.185.128.28', #dynamic server. Will change 
#     'port': '3306',
#     'username': 'thogativ_ab',
#     'password': 'Arun!23',
#     'database': 'thogativ_sthali',
# #     'driver': 'SQL Server Native Client 11.0',
#     'trusted_connection': 'yes',  
#     'legacy_schema_aliasing': False
# }
DB = {
    'drivername': 'mysql',
    'servername': 'localhost', #dynamic server. Will change 
    'port': '3306',
    'username': 'root',
    'password': 'root',
    'database': 'stocks',
#     'driver': 'SQL Server Native Client 11.0',
    'trusted_connection': 'yes',  
    'legacy_schema_aliasing': False
}

db_engine = create_engine(DB['drivername'] + '://' + DB['username'] +':'+ DB['password'] +
                       '@'+ DB['servername'] +':'+ DB['port']+ '/' + DB['database']+'?charset=utf8', encoding="utf-8")  
# df.read_sql('nse', db_engine, if_exists='append', index=False )
def getbhavbydate(datestr):
    driver.get("https://www.nseindia.com/products/content/equities/equities/archieve_eq.htm")
    report_eleme = Select(driver.find_element_by_id('h_filetype'))
    report_eleme.select_by_visible_text('Bhavcopy')
    datefield =  driver.find_element_by_id('date')
    datefield.send_keys(datestr)
    time.sleep(2)
    # datefield.perform()
    dayfield=driver.find_elements(By.CLASS_NAME, '  ui-datepicker-current-day')
    dayfield[0].click()

    time.sleep(1)

    getdata=driver.find_elements(By.CLASS_NAME, 'getdata-button')

    getdata[0].click()
    time.sleep(2)
    try:
        filelink=driver.find_element_by_xpath("//table/tbody/tr/td/a")
        filelink.click()
        return True
    except:
        print("File doen't exists")
        return False

def save2dbbydate(file):
#     print("File: {}".format(file))
    time.sleep(2)
    try:
        print("File: {}".format(file))
        zip_ref = ZipFile("D:/SoftwareWebApps/Python/python_wb/Stocks/Data/NSE/"+file, 'r')
        with zip_ref as my_zip_file:
            with my_zip_file.open(my_zip_file.namelist()[0]) as myZip:
                df=pd.read_csv(myZip)
                if 'Unnamed: 13' in df.columns:
                    df=df.drop(['Unnamed: 13'], axis=1)
                tstamp=datetime.strptime(df['TIMESTAMP'].iloc[0],'%d-%b-%Y')
                df['TIMESTAMP']=tstamp
                df.to_sql('nse', db_engine, if_exists='append', index=False )

    except:
#         print("Either file doesnt exist  or Zip file not good for file: {}".format(file))
        save2dbbydate(file)

In [11]:
file="cm19Feb2018bhav.csv.zip"
zip_ref = ZipFile("D:/SoftwareWebApps/Python/python_wb/Stocks/Data/NSE/"+file, 'r')

In [12]:
dtthen=dt.datetime.now()
stock_data=pd.read_sql_query('SELECT distinct TIMESTAMP FROM nse where 1 ORDER BY TIMESTAMP', db_engine)
# stock_data=pd.read_sql_query('SELECT * FROM `nse` WHERE 1', db_engine)

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "D:/SoftwareWebApps/Python/python_wb/Stocks/Data/NSE"}
# chromeOptions.add_argument("--headless") 
chromeOptions.add_experimental_option("prefs",prefs)
chromedriver = "D:/Installations/chromedriver.exe"
driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)

recentdate=datetime.now().toordinal()
# olddate=date(2018, 1, 19).toordinal()   #old from 2007, 12, 5 to 2007, 6, 5 done
olddate=stock_data['TIMESTAMP'].iloc[-1].toordinal()+1
for i in range(olddate,recentdate+1,1): #728110
    dtcurrent=dt.datetime.now()
    d = date.fromordinal(i)    
    if (d.strftime('%a')!='Sun'):
        print(d.strftime("%d-%m-%Y %a"))
        isfileexist=getbhavbydate(d.strftime("%d-%m-%Y"))
#         time.sleep(3)
#         d = date.fromordinal(olddate)  
        if(isfileexist):
            save2dbbydate(d.strftime("cm%d%b%Ybhav.csv.zip"))
        print("time took to enter data in db is {}".format(dt.datetime.now()-dtcurrent))
print(dt.datetime.now()-dtthen)
driver.close()

17-02-2018 Sat
File doen't exists
time took to enter data in db is 0:00:10.800917
19-02-2018 Mon
File: cm19Feb2018bhav.csv.zip
time took to enter data in db is 0:00:08.311669
20-02-2018 Tue
File: cm20Feb2018bhav.csv.zip
time took to enter data in db is 0:00:09.023148
21-02-2018 Wed
File: cm21Feb2018bhav.csv.zip
time took to enter data in db is 0:00:09.020797
22-02-2018 Thu
File: cm22Feb2018bhav.csv.zip
time took to enter data in db is 0:00:07.962184
23-02-2018 Fri
File: cm23Feb2018bhav.csv.zip
time took to enter data in db is 0:00:09.070475
24-02-2018 Sat
File doen't exists
time took to enter data in db is 0:00:05.632847
26-02-2018 Mon
File: cm26Feb2018bhav.csv.zip
time took to enter data in db is 0:00:07.856840
27-02-2018 Tue
File: cm27Feb2018bhav.csv.zip
time took to enter data in db is 0:00:08.013627
28-02-2018 Wed
File: cm28Feb2018bhav.csv.zip
time took to enter data in db is 0:00:23.369009
01-03-2018 Thu
File: cm01Mar2018bhav.csv.zip
time took to enter data in db is 0:00:07.949336

In [45]:
# stock_data=pd.read_sql_query('SELECT distinct TIMESTAMP FROM nse where 1 ORDER BY TIMESTAMP', db_engine)
# stock_data.tail(1)
date.fromordinal(recentdate).strftime("cm%d%b%Ybhav.csv.zip")

'cm19Feb2018bhav.csv.zip'

In [41]:
stock_data.tail(1)['TIMESTAMP'].iloc[0].strftime("%d-%m-%Y")
stock_data.tail(1)['TIMESTAMP'].iloc[0].toordinal()
stock_data['TIMESTAMP'].iloc[-1].toordinal()

AttributeError: 'int' object has no attribute 'strftime'

'cm09Feb2018bhav.csv.zip'